# TrainLB

In [1]:
using AutomotiveDrivingModels
using DynamicBayesianNetworkBehaviors

## Parameters

In [2]:
include(Pkg.dir("AutomotiveDrivingModels", "scripts", "extract_params.jl"))
dset_filepath_modifier = "_following" # _freeflow, _following, _lanechange
dset = JLD.load(joinpath(EVALUATION_DIR, "dataset2" * dset_filepath_modifier * ".jld"),
                "model_training_data")::ModelTrainingData2
runlogs_original = load_runlogs(dset)
streetnets = load_streetnets(runlogs_original)

fold = 1
cv_split_outer = get_fold_assignment_across_drives(dset, N_FOLDS)
cv_split_inner = drop_fold!(deepcopy(cv_split_outer), fold)
@assert(cv_split_inner.nfolds > 0)

ntraces = length(cv_split_outer.seg_assignment);

## Optimize Hyperparams

In [3]:
train_def = BehaviorTrainDefinition(
                                    LB_TrainParams(
                                        indicators=INDICATOR_SET2,
                                        max_parents=10,
                                        verbosity=1,
                                    ),
                                    BehaviorParameter[
                                    ])

preallocated_data = preallocate_learning_data(dset, train_def.trainparams)
print(train_def); sleep(0.1)

BehaviorTrainDefinition
LB Train Params
	targets: (:f_des_angle,:f_accel)
	indicators: [:posFyaw,:posFt,:speed,:velBx,:velBy,:velFs,:velFt,:turnrate,:acc,:accFs,:accFt,:accBx,:accBy,:d_ml,:d_mr,:ttcr_ml,:ttcr_mr,:est_ttcr,:a_req_stayinlane,:n_lane_left,:n_lane_right,:has_lane_right,:has_lane_left,:curvature,:past_1_f_accel,:past_2_f_accel,:past_3_f_accel,:past_1_f_des_angle,:past_2_f_des_angle,:past_3_f_des_angle]
	ridge_regression_constant: 0.0
	min_σ_lat:      0.0001
	min_σ_lon:      1.0e-5
	max_parents:    10

hyperparams:


In [4]:
preallocated_data.features_disc

4-element Array{AutomotiveDrivingModels.FeaturesNew.AbstractFeature,1}:
 AutomotiveDrivingModels.FeaturesNew.Feature_N_Lane_Left()   
 AutomotiveDrivingModels.FeaturesNew.Feature_N_Lane_Right()  
 AutomotiveDrivingModels.FeaturesNew.Feature_Has_Lane_Right()
 AutomotiveDrivingModels.FeaturesNew.Feature_Has_Lane_Left() 

In [5]:
preallocated_data.features_cont

16-element Array{AutomotiveDrivingModels.FeaturesNew.AbstractFeature,1}:
 AutomotiveDrivingModels.FeaturesNew.Feature_PosFyaw()         
 AutomotiveDrivingModels.FeaturesNew.Feature_PosFt()           
 AutomotiveDrivingModels.FeaturesNew.Feature_Speed()           
 AutomotiveDrivingModels.FeaturesNew.Feature_VelBx()           
 AutomotiveDrivingModels.FeaturesNew.Feature_VelBy()           
 AutomotiveDrivingModels.FeaturesNew.Feature_VelFs()           
 AutomotiveDrivingModels.FeaturesNew.Feature_VelFt()           
 AutomotiveDrivingModels.FeaturesNew.Feature_TurnRate()        
 AutomotiveDrivingModels.FeaturesNew.Feature_Acc()             
 AutomotiveDrivingModels.FeaturesNew.Feature_AccFs()           
 AutomotiveDrivingModels.FeaturesNew.Feature_AccFt()           
 AutomotiveDrivingModels.FeaturesNew.Feature_AccBx()           
 AutomotiveDrivingModels.FeaturesNew.Feature_AccBy()           
 AutomotiveDrivingModels.FeaturesNew.Feature_MarkerDist_Left() 
 AutomotiveDrivingModels.Featur

In [6]:
preallocated_data.features_nann

10-element Array{AutomotiveDrivingModels.FeaturesNew.AbstractFeature,1}:
 AutomotiveDrivingModels.FeaturesNew.Feature_TimeToCrossing_Left()        
 AutomotiveDrivingModels.FeaturesNew.Feature_TimeToCrossing_Right()       
 AutomotiveDrivingModels.FeaturesNew.Feature_EstimatedTimeToLaneCrossing()
 AutomotiveDrivingModels.FeaturesNew.Feature_A_REQ_StayInLane()           
 AutomotiveDrivingModels.FeaturesNew.Feature_Past{:f_accel,1}()           
 AutomotiveDrivingModels.FeaturesNew.Feature_Past{:f_accel,2}()           
 AutomotiveDrivingModels.FeaturesNew.Feature_Past{:f_accel,3}()           
 AutomotiveDrivingModels.FeaturesNew.Feature_Past{:f_des_angle,1}()       
 AutomotiveDrivingModels.FeaturesNew.Feature_Past{:f_des_angle,2}()       
 AutomotiveDrivingModels.FeaturesNew.Feature_Past{:f_des_angle,3}()       

In [7]:
# AutomotiveDrivingModels.optimize_hyperparams_cyclic_coordinate_ascent!(
#                     train_def, dset, preallocated_data, cv_split_inner)
# print(train_def); sleep(0.1)

## Train

In [8]:
LB = train(dset, preallocated_data, train_def.trainparams, fold, cv_split_outer, true)
sleep(0.5)
println("\n\n")
print(LB)
sleep(0.5)

_greedy_hillclimb_iter_on_node: 
Node In Training:
	index:            1
	target_as_parent: 0
	parents_disc:     Int64[]
	parents_cont:     Int64[]
	parents_nann:     Int64[]

trying disc     1  -> Δlogl =    97.190625
trying disc     2  -> Δlogl =    74.245801
trying disc     3  -> Δlogl =    33.089459
trying disc     4  -> Δlogl =    69.875048
trying cont     1  -> Δlogl =  1272.316498
trying cont     2  -> Δlogl =    -0.700964
trying cont     3  -> Δlogl =     9.078856
trying cont     4  -> Δlogl =     9.078853
trying cont     5  -> Δlogl =    47.975434
trying cont     6  -> Δlogl =     9.143832
trying cont     7  -> Δlogl =  1260.891702
trying cont     8  -> Δlogl =    46.868392
trying cont     9  -> Δlogl =    -1.660664
trying cont    10  -> Δlogl =    -3.926264
trying cont    11  -> Δlogl =    93.627273
trying cont    12  -> Δlogl =    -1.660640
trying cont    13  -> Δlogl =   105.747016
trying cont    14  -> Δlogl =     5.393566
trying cont    15  -> Δlogl =     1.644515
trying c

## Likelihood

In [26]:
arr_logl_train = Float64[]
arr_logl_test = Float64[]
for frameind in 1 : nrow(dset.dataframe)
    logl = calc_action_loglikelihood(LB, dset.dataframe, frameind)
    
    if is_in_fold(fold, cv_split_outer.frame_assignment[frameind], true)
        push!(arr_logl_test, logl)
    else
        push!(arr_logl_train, logl)
    end
end

@printf("%12s  %10s  %10s  %10s\n", "min", "mean", "median", "max")
@printf("%12.4f  %10.4f  %10.4f  %10.4f\n", minimum(arr_logl_train), mean(arr_logl_train), median(arr_logl_train), maximum(arr_logl_train))
@printf("%12.4f  %10.4f  %10.4f  %10.4f\n", minimum(arr_logl_test), mean(arr_logl_test), median(arr_logl_test), maximum(arr_logl_test))

         min        mean      median         max
  -1573.3771      3.7556      4.0916      5.5212
   -283.1268      3.9679      4.0882      5.4385
